In [1]:
import sys
sys.path.append('../')
import pandas as pd
import pickle

from lib.rmodel import Dataset, BertClassifier, train, predict

In [2]:
datapath = f'../while_waiting_for_store.csv'
df = pd.read_csv(datapath)
df=df[df.columns[2:]]
df.head()
df = df.fillna(value=str(''))
df['text'] = df['title'] + '  ' + df['selftext']
df['text'] = df['text'].fillna(value='').astype(str)
df_clean = df[df.relevance!=-1]


In [3]:
model = pickle.load(open('../model/bert.pckl','rb'))

In [ ]:
df_pred = df_clean['text'].copy()
pred = predict(model, df_pred)

In [ ]:
pred.sort_values('relevance', ascending=False)